In [1]:
#importazione dei dati ------------------------------------------------------------------------------------------------------------

from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.linalg import toeplitz
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate

df = pd.read_csv (r'/Users/patrickpinta/Desktop/1datasetpanda/soybean.csv',header=0,error_bad_lines=False) 

#modellazione dei dati training set------------------------------------------------------------------------------------------------------------

xExpUsa= df.iloc[0:334,1:2]
xExpUsa = np.array(xExpUsa)
xExpBra=df.iloc[0:334,3:4]
xExpBra = np.array(xExpBra)
xExpArg=df.iloc[0:334,5:6]
xExpArg = np.array(xExpArg)
xImportChina=xExpUsa+xExpBra+xExpArg


x=df.iloc[0:334,0:1]
x2=df.iloc[0:334,2:3]
x3=df.iloc[0:334,4:5]
x4=df.iloc[0:334,6:7]
x5=df.iloc[0:334,8:11]
x6=df.iloc[0:334,15:17]



xImportChina=xImportChina.tolist()
xImportChina = DataFrame (xImportChina,columns=['ImportChina'])
x=x.join(xImportChina)
x=x.join(x2)
x=x.join(x3)
x=x.join(x4)
x=x.join(x5)
x=x.join(x6)



y= df.iloc[0:334,23:24]


#stardadizzazione dei dati training set------------------------------------------------------------------------------------------------------------

scaler = MinMaxScaler(feature_range=(-1,1))

x_train_stand=(scaler.fit_transform(x))

#modello dati training set------------------------------------------------------------------------------------------------------------    
    
x_train_stand, y = np.array(x_train_stand), np.array(y)
x_train_stand = sm.add_constant(x_train_stand)

#procedura per ols model ------------------------------------------------------------------------------------------------------------

ols_model = sm.OLS(y, x_train_stand)
ols_results = ols_model.fit(cov_type='HC1') 
print(ols_results.summary())

reg = linear_model.LinearRegression()
reg= reg.fit(x_train_stand,y)

#forecast ------------------------------------------------------------------------------------------------------------

ypredTrain = reg.predict(x_train_stand)

print('r2 train: ',r2_score(y,ypredTrain))
print('mse train: ',mean_squared_error(y,ypredTrain))

scores=cross_validate(reg, x_train_stand, y, cv=5,scoring=('r2', 'neg_mean_squared_error'),return_train_score=True)
print('r2 cv: ', scores['test_r2'].mean())
print('mse cv: ', abs(scores['test_neg_mean_squared_error'].mean()))






                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.645
Model:                            OLS   Adj. R-squared:                  0.634
Method:                 Least Squares   F-statistic:                     54.70
Date:                Fri, 14 May 2021   Prob (F-statistic):           1.53e-63
Time:                        15:53:58   Log-Likelihood:                -2055.1
No. Observations:                 334   AIC:                             4132.
Df Residuals:                     323   BIC:                             4174.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        911.7931     58.165     15.676      0.0

### 